<a href="https://colab.research.google.com/github/MilanBinsMathew/Yolo_V5_Pen_Detection/blob/main/IMGAUG_YOLOv5_PenDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Pen-Detection using YOLOv5**

This project aims to detect pens in an image using the YOLO V5 model

## 1. Setup the Environment

Clone the YOLO v5 model developed by Ultralytics.
Then install the required dependencies and frameworks for the project.

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

import torch
import utils
display = utils.notebook_init()

YOLOv5 🚀 v6.2-34-ge0700cc Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 37.4/78.2 GB disk)


## Import Dataset

The dataset used for the project has been custom sourced and contains around 283 images for training and another 40 images for testing and validation respectively.

The .zip file containing the images and labels in YOLO format are uploaded and then unzipped to the required directory.

In [ ]:
!ls 

best.pt		 detect.py   models	       train.py
classify	 export.py   README.md	       tutorial.ipynb
CONTRIBUTING.md  hubconf.py  requirements.txt  utils
data		 LICENSE     setup.cfg	       val.py


In [ ]:
!unzip -q ../Pen_yolo.zip -d ../ 

## 2. Image Augmentation

Augmenting the images by rotating each image form 0 to 350 degrees with increments of 10 degrees on the pen and bounding box

In [ ]:
import numpy as np
import imageio
import os
import math

%pip install git+https://github.com/aleju/imgaug.git
import imgaug
import imgaug.augmenters as iaa
import imgaug.parameters as iap
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/aleju/imgaug.git to /tmp/pip-req-build-53e0l7hi
  Running command git clone -q https://github.com/aleju/imgaug.git /tmp/pip-req-build-53e0l7hi


In [ ]:
len(os.listdir("/content/Pen_yolo/train/images"))

283

In [ ]:

rotations = [x for x in range(0,351,10)]
augment = iaa.Sequential(iaa.Affine(rotate = iap.DeterministicList(rotations)))


In [ ]:
n=len(os.listdir('/content/Pen_yolo/train/images'))
image_files = os.listdir('/content/Pen_yolo/train/images')
label_files = lab_var = [[a for a in open('/content/Pen_yolo/train/labels/' +x).readline().split()] for x in os.listdir("/content/Pen_yolo/train/labels")]
for i in range(n//2):
  
  image = imageio.imread('/content/Pen_yolo/train/images/'+image_files[i])
  i_h = image.shape[0]
  i_w = image.shape[1]
  y_x = float(label_files[i][1])*i_w
  y_y = float(label_files[i][2])*i_h
  y_w = float(label_files[i][3])*i_w
  y_h = float(label_files[i][4])*i_h
  x1, y1 = y_x - y_w/2, y_y - y_h/2
  x2, y2 = y_x + y_w/2, y_y + y_h/2
  label = [BoundingBox(x1=x1,x2=x2,y1=y1,y2=y2)]

 
  imglist = [image for i in range(len(rotations))]
  
  
  
  for j in range(0,len(rotations)):
    rads = math.radians(rotations[j])
    n_h = y_w*abs(math.sin(rads)) + y_h*abs(math.cos(rads))
    n_w = y_w*abs(math.cos(rads)) + y_h*abs(math.sin(rads))
    x1, y1 = y_x - n_w/2, y_y - n_h/2
    x2, y2 = y_x + n_w/2, y_y + n_h/2
    label.append(BoundingBox(x1=x1,x2=x2,y1=y1,y2=y2))

  labels = BoundingBoxesOnImage(label, shape = (image.shape[0],image.shape[1]))
  image_aug,aug_labels = augment(images = imglist, bounding_boxes = labels)
  
  path = "/content/Pen_yolo/train/"
  for j in range(1,len(rotations)):
    file_name = path+"images/image"+str(i)+str(j)
    text_file_name = path+"labels/image"+str(i)+str(j)
    imageio.imwrite(file_name+".jpg", image_aug[j])
    with open(text_file_name+".txt",'w') as f:
      f.write("0 " + str(label[j].center_x/image_aug[j].shape[0]) +" "+ str(label[j].center_y/image_aug[j].shape[1]) +" "+ str(label[j].width/image_aug[j].shape[0]) +" "+ str(label[j].height/image_aug[j].shape[1]))




In [ ]:
print(len(os.listdir('/content/Pen_yolo/train/images')))
print(len(os.listdir('/content/Pen_yolo/train/labels')))

4919
4919


In [ ]:
print("0 " + str(label[0].center_x) +" "+ str(label[0].center_y) +" "+ str(label[0].width) +" "+ str(label[0].height))


In [ ]:
'''
path = "/content/Pen_yolo/train/"
for i in range(1,len(rotations)):
  file_name = path+"images/image"+str(i)
  text_file_name = path+"labels/image"+str(i)
  imageio.imwrite(file_name+".jpg", image_aug[i])
  with open(text_file_name+".txt",'w') as f:
    f.write("0 " + str(label[i].center_x/image_aug[i].shape[0]) +" "+ str(label[i].center_y/image_aug[i].shape[1]) +" "+ str(label[i].width/image_aug[i].shape[0]) +" "+ str(label[i].height/image_aug[i].shape[1]))'''

## 2. Training

Train the model with respect to the given data. 
Note arguments given may be modified depending upon the needs and requirements.

In [ ]:

!python train.py --img 640 --batch 8 --epochs 75 --data custom_data.yaml --weights best.pt --cache

train: weights=best.pt, cfg=, data=custom_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=75, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-34-ge0700cc Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, d

## 3. Detection

Live detection in a video using Yolo-V5
The video must be uploaded to the runtime and the argument (source path) must be modified accordingly.

In [ ]:
!python detect.py --weights runs/train/exp2/weights/best.pt --img 640 --conf 0.4 --source ../pen_test.mp4

detect: weights=['runs/train/exp2/weights/best.pt'], source=../pen_test.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.2-34-ge0700cc Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/615) /content/pen_test.mp4: 640x384 1 pen, 11.7ms
video 1/1 (2/615) /content/pen_test.mp4: 640x384 1 pen, 9.3ms
video 1/1 (3/615) /content/pen_test.mp4: 640x384 1 pen, 9.3ms
video 1/1 (4/615) /content/pen_test.mp4: 640x384 1 pen, 9.2ms
video 1/1 (5/615) /content/pen_test.mp4: 640x384 1 pen, 9.2ms
video 1/1 (6/615) /content/pen_test.mp4: 640x384 1 pen